In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from lifelines import CoxPHFitter
from datautils.dataset import Dataset

In [ ]:
# dataset = Dataset("data/sepsis_data",
#                     batchSize=100,
#                     train_ratio=0.8,
#                     normalize=True,
#                     padding=False,
#                     imputeForward=False)

In [61]:
patient_df = pd.read_csv('data/temp_hazard/allpatients_forward_mean_imputed').drop(columns=['Unnamed: 0','Unit1','Unit2'])

In [62]:
mean_vals = patient_df.loc[:,'HR':'Platelets'].astype(np.float64).mean()
patient_df.fillna(mean_vals, inplace=True)
patient_df[patient_df.pid=="p00001"].head()
patient_df.shape
patient_df.HospAdmTime.unique().shape

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,83.855364,97.046165,36.797649,126.552435,86.730349,66.634428,18.653171,33.481988,-3.757377,...,10.506372,40.006666,10.499074,291.835891,199.84959,73,1,-214.64,1,0
179973,p00001,93.000000,92.500000,36.797649,110.000000,76.000000,56.000000,22.000000,33.481988,-3.757377,...,10.506372,40.006666,10.499074,291.835891,199.84959,73,1,-214.64,2,0
179974,p00001,91.000000,96.000000,36.797649,108.000000,84.500000,72.000000,23.500000,33.481988,-3.757377,...,10.506372,40.006666,10.499074,291.835891,199.84959,73,1,-214.64,3,0
179975,p00001,93.000000,98.000000,36.797649,123.000000,87.000000,61.000000,21.000000,33.481988,-3.757377,...,10.506372,40.006666,10.499074,291.835891,199.84959,73,1,-214.64,4,0
179976,p00001,93.000000,95.000000,36.797649,110.000000,81.000000,70.000000,20.000000,33.481988,-3.757377,...,10.506372,40.006666,10.499074,291.835891,199.84959,73,1,-214.64,5,0


(188453, 40)

(2976,)

In [426]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0','Unit1','Unit2'])

In [427]:
patient_df[patient_df.pid=="p00001"].head()
patient_df.shape
patient_df.HospAdmTime.unique().shape
patient_df[patient_df.SepsisLabel == 1].HospAdmTime.unique()

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
179972,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,1,0
179973,p00001,93.0,92.5,36.5,110.0,76.0,56.0,22.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,2,0
179974,p00001,91.0,96.0,36.5,108.0,84.5,72.0,23.5,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,3,0
179975,p00001,93.0,98.0,36.5,123.0,87.0,61.0,21.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,4,0
179976,p00001,93.0,95.0,36.5,110.0,81.0,70.0,20.0,33.517515,-3.160918,...,11.3,40.80889,10.8,296.932741,170.0,73,1,-214.64,5,0


(188453, 40)

(2976,)

array([ 0.00000e+00, -3.10000e-01, -5.39000e+01, -4.91000e+00,
       -5.00000e-02, -2.14600e+01, -1.32730e+02, -3.00470e+02,
       -1.92270e+02, -3.04000e+01, -8.01000e+00, -5.28000e+00,
       -2.43200e+01, -2.85800e+01, -4.91800e+01, -5.93740e+02,
       -2.81780e+02, -4.28000e+00, -3.53000e+01, -9.93000e+00,
       -1.20000e-01, -2.00000e-02, -5.63820e+02, -3.50260e+02,
       -3.19850e+02, -9.21000e+00, -6.03400e+01, -5.27240e+02,
       -2.47000e+00, -1.17172e+03, -7.52000e+00, -3.08740e+02,
       -1.15000e+00, -3.00000e-02, -2.60000e+00, -7.00000e-02,
       -2.52300e+01, -1.00000e-02, -7.76200e+01, -1.97780e+02,
       -7.08400e+01, -7.29000e+00, -2.58200e+01, -3.31000e+00,
       -2.66400e+01, -3.58800e+01, -9.75700e+01, -2.51100e+01,
       -2.92000e+00, -2.34000e+00, -1.01100e+01, -1.10000e-01,
       -1.19900e+01, -4.40000e-01, -9.07000e+00, -1.70000e-01,
       -2.69600e+01, -3.64940e+02, -7.21200e+01, -4.95000e+00,
       -2.91000e+00, -4.00000e-02, -8.00000e-02, -3.300

In [130]:
#patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

In [63]:
patient_df[patient_df.pid=="p00001"].SepsisLabel.unique()

array([0], dtype=int64)

In [64]:
patient_df.pid.unique().shape

(5000,)

In [65]:
re_idx = patient_df.set_index(["pid","ICULOS"])
arr = re_idx.groupby("pid").apply(lambda x: x.values).values

train_raw, test_raw = train_test_split(arr, test_size=0.2)

In [66]:
train_raw[0].shape
train_raw[0][0]

(49, 38)

array([ 83.85536447,  97.046165  ,  36.7976492 , 126.55243472,
        86.73034903,  66.63442824,  18.65317139,  33.48198843,
        -3.7573769 ,  22.09433411,   0.46550563,   7.38880276,
        39.54079435,  96.55090276, 119.30067329,  22.3528374 ,
        91.18621187,   7.73577195, 105.98371503,   1.60705865,
         0.84776099, 132.09230487,   2.13049022,   2.04544313,
         3.4859577 ,   4.06307364,   1.4891377 ,   5.77915899,
        32.05628105,  10.50637232,  40.00666633,  10.49907376,
       291.83589079, 199.84958988,  61.        ,   1.        ,
        -0.3       ,   0.        ])

In [67]:
# create windowing system here
T = 6
#idx = 10
def process_data(df: pd.DataFrame, T: int) -> (pd.DataFrame, np.array):
    processed = []
    labels = []

    for idx in range(df.shape[0]):
        if df[idx][:,-1].sum() == 0:
            processed.extend([[row,7,1] for row in df[idx][:,:-1]])
        else:
            sepsis_count = 0
            for i in range(df[idx].shape[0]):
                t = (T + 1) - sepsis_count
                t = t if t >= 1 else 1
                s = 1 if t > T else 0
                processed.append([df[idx][i][:-1],t,s])
                sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
        labels.extend(df[idx][:,-1].tolist())
                
    return (pd.DataFrame(processed, columns=["x","t","s"]), np.array(labels))
# Naive windowing:
#             for i in range(df[idx].shape[0]):
#                 window = df[idx][i:i+T]
#                 matches = np.where(window[:,-1]==1)[0]
#                 if matches.size > 0:
#                     t = matches[0] + 1
#                     s = 0
#                 else:
#                     t = T + 1
#                     s = 1
#                 processed.append([df[idx][i][:-1],t,s])

# def test(df: pd.DataFrame, T: int) -> pd.DataFrame:
#     processed = []

#     #for idx in range(df.shape[0]):
#     idx = 10
#     if df[idx][:,-1].sum() == 0:
#         processed.extend([[row,7,1] for row in df[idx][:,:-1]])
#     else:
#         sepsis_count = 0
#         for i in range(df[idx].shape[0]):
#             t = (T + 1) - sepsis_count
#             t = t if t >= 1 else 1
#             s = 1 if t > T else 0
#             processed.append([df[idx][i][:-1],t,s])
#             sepsis_count += 1 if df[idx][i][-1] == 1 else 0
                
#     return pd.DataFrame(processed, columns=["x","t","s"])

# np.array(train[idx][:,:-1]).shape
# np.repeat([[7,1]],train[idx].shape[0],axis=0).shape
# t_s = np.repeat([[7,1]],train[idx].shape[0],axis=0)
# t_s
# x_idx = train[idx][:,:-1].reshape(train[idx][:,:-1].shape[0],1,-1)
# x_idx.shape
# processed_idx = np.concatenate((x_idx,t_s),axis=1)
# processed_idx.shape
#if train[idx][:,-1].sum() == 0:
# a = np.ones((4,1)) * 3
# b = np.repeat([[7,1]],4,axis=0)    
# a
# b
# np.hstack((a,b))
#pd.DataFrame([[train[idx][i][:-1],t,s]], columns=["x","t","s"])
# for idx in range(4000):
#     #window = train[idx][i:i+T]
#     if train[idx][:,-1].sum() > 0:
#         break

In [68]:
X_train, y_train = process_data(train_raw, T)
X_test, y_test = process_data(test_raw, T)

In [69]:
X_train.head()

,x,t,s
0,"[83.85536447147673, 97.04616499808523, 36.7976...",7,1
1,"[73.5, 99.5, 36.79764920289012, 157.0, 107.0, ...",7,1
2,"[75.0, 98.0, 36.79764920289012, 158.0, 113.0, ...",7,1
3,"[77.0, 100.0, 36.79764920289012, 155.0, 105.0,...",7,1
4,"[82.0, 100.0, 36.79764920289012, 160.0, 88.0, ...",7,1


In [70]:
inverse_s = 1-X_train.s
X_train_cph = pd.DataFrame(X_train.x.values.tolist(), columns=patient_df.columns[1:-2])
X_train_cph["s"] = inverse_s
X_train_cph["w"] = (inverse_s * 70) + X_train.s
X_train_cph["t"] = X_train.t

In [71]:
cph = CoxPHFitter(penalizer=0.2)
cph.fit(X_train_cph, duration_col='t', event_col='s', weights_col='w', step_size=0.070, show_progress=True, robust=False)

Iteration 1: norm_delta = 0.04777, step_size = 0.0700, ll = -1643199.00616, newton_decrement = 23239.90719, seconds_since_start = 0.1
Iteration 2: norm_delta = 0.04351, step_size = 0.0700, ll = -1640061.11540, newton_decrement = 19139.67232, seconds_since_start = 0.2
Iteration 3: norm_delta = 0.03996, step_size = 0.0700, ll = -1637476.54266, newton_decrement = 15912.68159, seconds_since_start = 0.4
Iteration 4: norm_delta = 0.04426, step_size = 0.0840, ll = -1635327.55606, newton_decrement = 13319.96019, seconds_since_start = 0.5
Iteration 5: norm_delta = 0.03951, step_size = 0.0823, ll = -1633184.78452, newton_decrement = 10806.64805, seconds_since_start = 0.7
Iteration 6: norm_delta = 0.03544, step_size = 0.0807, ll = -1631479.43749, newton_decrement = 8855.27400, seconds_since_start = 0.8
Iteration 7: norm_delta = 0.03904, step_size = 0.0968, ll = -1630108.70351, newton_decrement = 7316.47402, seconds_since_start = 1.0
Iteration 8: norm_delta = 0.03452, step_size = 0.0949, ll = -162

<lifelines.CoxPHFitter: fitted with 150470 observations, 148556 censored>

In [72]:
#cph.check_assumptions(X_train_cph,show_plots=False,plot_n_bootstraps=0)
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 150470 observations, 148556 censored>
      duration col = 't'
         event col = 's'
       weights col = 'w'
         penalizer = 0.2
number of subjects = 150470
  number of events = 1914
    log-likelihood = -1623236.62
  time fit was run = 2019-04-04 06:44:44 UTC

---
                  coef  exp(coef)  se(coef)      z      p  -log2(p)  lower 0.95  upper 0.95
HR                0.01       1.01      0.00  72.18 <0.005       inf        0.01        0.01
O2Sat             0.01       1.01      0.00   7.72 <0.005     46.28        0.01        0.01
Temp              0.18       1.20      0.00  48.58 <0.005       inf        0.17        0.19
SBP              -0.00       1.00      0.00 -18.82 <0.005    260.08       -0.00       -0.00
MAP               0.00       1.00      0.00   2.22   0.03      5.26        0.00        0.00
DBP              -0.01       0.99      0.00 -15.53 <0.005    178.26       -0.01       -0.01
Resp              0.00       1.00      0.00  

In [73]:
train_preds = 1-cph.predict_survival_function(X_train_cph.drop(columns=["s","t"]),times=[6])

In [74]:
lst = [1 if i >=0.5 else 0 for i in train_preds]
((lst == y_train).sum() / y_train.shape[0]) * 100
roc_auc_score(y_train, train_preds)

82.07682594537117

0.7061080007484359

In [75]:
train_preds2 = cph.predict_cumulative_hazard(X_train_cph.drop(columns=["s","t"]),times=[6])

In [76]:
lst = [1 if i >=0.5 else 0 for i in train_preds2]
((lst == y_train).sum() / y_train.shape[0]) * 100
roc_auc_score(y_train, train_preds2)

59.18920715092709

0.7061080007484359

In [77]:
train_preds3 = cph.predict_partial_hazard(X_train_cph.drop(columns=["s","t"]))
lst = [1 if i >=0.5 else 0 for i in train_preds3]
((lst == y_train).sum() / y_train.shape[0]) * 100
roc_auc_score(y_train, train_preds3)

98.57646042400478

0.7061080007484359

In [79]:
X_test_cph = pd.DataFrame(X_test.x.values.tolist(), columns=patient_df.columns[1:-2])

In [80]:
test_preds = 1-cph.predict_survival_function(X_test_cph,times=[6])
lst = [1 if i >=0.5 else 0 for i in test_preds]
((lst == y_test).sum() / y_test.shape[0]) * 100
round(roc_auc_score(y_test, test_preds), 3)

80.01737619461338

0.673

In [81]:
def get_metrics(ty, py, threshold=0.5):
    print('-'*20)
    auc = roc_auc_score(ty, py)
    print(f"AUC = {auc}")
    c_m = confusion_matrix(ty, np.array(py > threshold).astype(int))
    print(c_m)
    PPV = c_m[1,1] / (c_m[1,1] + c_m[0,1])
    print(f"PPV/Precision = {PPV}")
    TPR = c_m[1,1] / c_m[1].sum()
    print(f"TPR/Sensitivity/Recall = {TPR}")
    TNR = c_m[0,0] / c_m[0].sum()
    print(f"TNR/Specificity = {TNR}")

In [83]:
get_metrics(y_test, test_preds, threshold=0.5)

--------------------
AUC = 0.6725981960102807
[[30171  7331]
 [  259   222]]
PPV/Precision = 0.029392294452535415
TPR/Sensitivity/Recall = 0.46153846153846156
TNR/Specificity = 0.8045170924217375


In [445]:
# test simple cphfitter from lifelines

# implement actual cox-weibull from "scratch"?